# 3 Teoría de la estimación
## 3.4. Regresión logística

<p align="right">
Autor: Emmanuel Alcalá
<br>
<a href="https://scholar.google.com.mx/citations?hl=en&user=3URusCgAAAAJ&view_op=list_works&sortby=pubdate">Google Scholar</a>
</p>

<p align="left">
<br>
<a href="https://jealcalat.github.io/Analisis_multivariado/">Regresar a la página del curso</a>
</p>

---


LASSO (Least Absolute Shrinkage and Selection Operator) es una técnica de regularización que no solo reduce el sobreajuste sino que también puede ayudar en la selección de características, ya que puede reducir ciertos coeficientes a cero, eliminando esencialmente la característica del modelo.


In [9]:
# Importando las bibliotecas necesarias
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Cargando el dataset
data = load_breast_cancer()
X = data.data
y = data.target

# Dividiendo los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Es importante escalar los datos cuando se utiliza LASSO
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenando el modelo de regresión logística con regularización L1 (LASSO)
clf = LogisticRegression(penalty='l1', solver='saga', max_iter=10000)
clf.fit(X_train_scaled, y_train)

# Coeficientes del modelo
coef = clf.coef_

# Características seleccionadas
selected_features = np.where(coef != 0)[1]
print(f"Características seleccionadas con LASSO: {selected_features}")
# imprimir nombres de las características seleccionadas
print(f"Nombre de las características seleccionadas con LASSO: {data.feature_names[selected_features]}")


Características seleccionadas con LASSO: [ 7  8 10 11 14 15 18 19 20 21 23 24 26 27 28]
Nombre de las características seleccionadas con LASSO: ['mean concave points' 'mean symmetry' 'radius error' 'texture error'
 'smoothness error' 'compactness error' 'symmetry error'
 'fractal dimension error' 'worst radius' 'worst texture' 'worst area'
 'worst smoothness' 'worst concavity' 'worst concave points'
 'worst symmetry']


El término `penalty` en regresión logística (y otros modelos lineales) se refiere a la adición de un término de regularización al objetivo de optimización del modelo. El objetivo de esta regularización es evitar el sobreajuste y, en el caso del penalty `l1`, lograr la selección de características al reducir algunos coeficientes a cero.

1. **Función Objetivo sin Regularización**:
   Sin regularización, la función objetivo de la regresión logística se centra únicamente en minimizar el error entre las predicciones y los valores verdaderos (la log-verosimilitud negativa).

2. **Introducción del Penalty `l1`**:
   Cuando introducimos el penalty `l1`, estamos añadiendo un término de regularización basado en la norma l1 de los coeficientes. La norma l1 es simplemente la suma de los valores absolutos de los coeficientes.
   
  $\text{Función objetivo} = \text{Log-verosimilitud negativa} + \lambda \sum | \beta_i | $
   
   Donde $\beta_i $ son los coeficientes del modelo y $\lambda $ es un parámetro de regularización, a menudo llamado `C` en sklearn, que controla la fuerza de la regularización (notar que en `sklearn`, valores más altos de `C` significan menos regularización).

3. **Selección de Características con Penalty `l1`**:
   La belleza del penalty `l1` radica en su capacidad para hacer exactamente cero algunos coeficientes. A medida que aumenta la fuerza de la regularización (es decir,$\lambda $ aumenta o `C` disminuye), más coeficientes son reducidos a cero. Coeficientes que son cero efectivamente eliminan esa característica del modelo, logrando así la selección de características. 

El uso del penalty `l1` es especialmente útil cuando se tiene un conjunto de datos con un gran número de características y se quiere reducir la dimensionalidad seleccionando solo las características más relevantes.

In [11]:
# Creando el nuevo conjunto de datos con las características seleccionadas
X_train_scaled_selected = X_train_scaled[:, selected_features]
X_test_scaled_selected = X_test_scaled[:, selected_features]

# Reentrenando el modelo con las características seleccionadas
clf_selected = LogisticRegression(solver='saga', max_iter=10000)
clf_selected.fit(X_train_scaled_selected, y_train)

# Evaluando el rendimiento con las características seleccionadas en el conjunto de prueba
score_selected = clf_selected.score(X_test_scaled_selected, y_test)
print(f"Precisión con las características seleccionadas en el conjunto de prueba: {score_selected:.2f}")


Precisión con las características seleccionadas en el conjunto de prueba: 0.97


¿Por qué es importante escalar los datos con LASSO?

La razón más importante es la sensibilidad a la magnitud de las características: Dado que LASSO impone una penalización basada en la magnitud de los coeficientes, características con diferentes escalas resultarán en coeficientes de diferentes magnitudes, incluso si tienen la misma importancia en la predicción. Una característica con una gran escala (por ejemplo, salario que podría estar en decenas de miles) tendrá un coeficiente más pequeño en comparación con una característica con una escala más pequeña (por ejemplo, edad que podría estar entre 0 y 100). La penalización en LASSO afectaría de manera desproporcionada al coeficiente con una escala más pequeña, simplemente debido a su magnitud, lo que podría llevar a la eliminación incorrecta de características importantes.